In [1]:
# !pip3 install -r https://gitlab.com/checkthat_lab/clef2024-checkthat-lab/-/raw/main/task1/requirements.txt

In [2]:
!pip install accelerate
!pip install transformers
!pip install datasets
!pip install jsonlines
!apt-get install wget

# IMPORT ALL YOUR LIBRARIES
# SUGGESTED LIBRARIES
import pandas as pd
import numpy as np
import torch
from sklearn.preprocessing import OneHotEncoder
from transformers import BertTokenizer, BertForSequenceClassification, AdamW, get_linear_schedule_with_warmup, TrainingArguments, Trainer, AutoTokenizer, DataCollatorWithPadding
from torch.utils.data import DataLoader, Dataset, RandomSampler, SequentialSampler
from tqdm import tqdm
import accelerate
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
import matplotlib.pyplot as plt
import requests
from io import StringIO

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 6.5 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

##Baselines

In [3]:
# Create directory if it doesn't exist
import os
directory = "dataset_converted"
os.mkdir(directory) if not os.path.exists(directory) else None

# Datasets URLs and names
datasets = [
    ("https://drive.google.com/uc?export=download&id=1hAV0MCS2i_m64BNdQddpAqDE5v7gu0Dp", "train_dataset.tsv"),
    ("https://drive.google.com/uc?export=download&id=1IrUkCm6FGWxTYv8btPtGMtXCtGiR26YQ", "dev_dataset.tsv"),
    ("https://drive.google.com/uc?export=download&id=1qUoA1MsLLyb9mIAOn_9LI9t9_En82diF", "test_dataset.tsv"),
    ("https://drive.google.com/uc?export=download&id=1UsrMqt6wAb9Tw7Xkw04tfqukYvjZWpre", "new_test.tsv"),
]

# Convert and save datasets
for url, filename in datasets:
    response = requests.get(url)
    response.raise_for_status()
    dataset = pd.read_csv(StringIO(response.text), sep="\t")
    dataset.to_csv(os.path.join(directory, filename), sep="\t", index=False)

print("Datasets converted and saved successfully!")

Datasets converted and saved successfully!


In [4]:
!mkdir format_checker
!wget -O format_checker/task_1.py https://gitlab.com/checkthat_lab/clef2024-checkthat-lab/-/raw/main/task1/format_checker/task_1.py

--2024-05-03 05:31:58--  https://gitlab.com/checkthat_lab/clef2024-checkthat-lab/-/raw/main/task1/format_checker/task_1.py
Resolving gitlab.com (gitlab.com)... 172.65.251.78, 2606:4700:90:0:f22e:fbec:5bed:a9b9
Connecting to gitlab.com (gitlab.com)|172.65.251.78|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2020 (2.0K) [text/plain]
Saving to: ‘format_checker/task_1.py’

format_checker/task 100%[===================>]   1.97K  --.-KB/s    in 0s      

2024-05-03 05:31:59 (29.3 MB/s) - ‘format_checker/task_1.py’ saved [2020/2020]



In [5]:
!mkdir scorer
!wget -O scorer/task_1.py https://gitlab.com/checkthat_lab/clef2024-checkthat-lab/-/raw/main/task1/scorer/task_1.py
!wget -O scorer/utils.py https://gitlab.com/checkthat_lab/clef2024-checkthat-lab/-/raw/main/task1/scorer/utils.py

--2024-05-03 05:31:59--  https://gitlab.com/checkthat_lab/clef2024-checkthat-lab/-/raw/main/task1/scorer/task_1.py
Resolving gitlab.com (gitlab.com)... 172.65.251.78, 2606:4700:90:0:f22e:fbec:5bed:a9b9
Connecting to gitlab.com (gitlab.com)|172.65.251.78|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4806 (4.7K) [text/plain]
Saving to: ‘scorer/task_1.py’

scorer/task_1.py    100%[===================>]   4.69K  --.-KB/s    in 0s      

2024-05-03 05:31:59 (70.7 MB/s) - ‘scorer/task_1.py’ saved [4806/4806]

--2024-05-03 05:31:59--  https://gitlab.com/checkthat_lab/clef2024-checkthat-lab/-/raw/main/task1/scorer/utils.py
Resolving gitlab.com (gitlab.com)... 172.65.251.78, 2606:4700:90:0:f22e:fbec:5bed:a9b9
Connecting to gitlab.com (gitlab.com)|172.65.251.78|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3892 (3.8K) [text/plain]
Saving to: ‘scorer/utils.py’

scorer/utils.py     100%[===================>]   3.80K  --.-KB/s    in 0s      



In [6]:
# !mkdir baselines
# !wget -O baselines/task_1.py https://gitlab.com/checkthat_lab/clef2024-checkthat-lab/-/raw/main/task1/baselines/task_1.py

In [7]:
os.makedirs("/content/data/", exist_ok=True)

In [8]:
# !python baselines/task_1.py --train-file-path=/content/dataset_converted/train_dataset.tsv --dev-file-path=/content/dataset_converted/dev_dataset.tsv -l english

##Majority Baseline for english F1 (positive class): 0.0

##Random Baseline for english F1 (positive class): 0.30647291941875826

##Ngram Baseline for english F1 (positive class): 0.8526077097505668

##Task 1: Loading the dataset using pandas

In [9]:
#TrainDataset
# Google Drive sharing URL
url_train = "https://drive.google.com/uc?export=download&id=1hAV0MCS2i_m64BNdQddpAqDE5v7gu0Dp"

# Make the request to Google Drive
response = requests.get(url_train)
response.raise_for_status()  # Check if the request was successful

# Convert text response to a pandas DataFrame
train_dataset = pd.read_csv(StringIO(response.text), sep="\t")

#DevDataset
# Google Drive direct download URL for dev_dataset
url_dev = "https://drive.google.com/uc?export=download&id=1IrUkCm6FGWxTYv8btPtGMtXCtGiR26YQ"

# Make the request to Google Drive
response = requests.get(url_dev)
response.raise_for_status()  # Check if the request was successful

# Convert text response to a pandas DataFrame
dev_dataset = pd.read_csv(StringIO(response.text), sep="\t")

#TestDataset
# Google Drive direct download URL for test dataset
url_test = "https://drive.google.com/uc?export=download&id=1qUoA1MsLLyb9mIAOn_9LI9t9_En82diF"

# Make the request to Google Drive for the test dataset
response_test = requests.get(url_test)
response_test.raise_for_status()  # Check if the request was successful

# Convert text response to a pandas DataFrame for the test dataset
test_dataset = pd.read_csv(StringIO(response_test.text), sep="\t")

#NewTest

url_test = "https://drive.google.com/uc?export=download&id=1UsrMqt6wAb9Tw7Xkw04tfqukYvjZWpre"

# Make the request to Google Drive for the test dataset
response_test = requests.get(url_test)
response_test.raise_for_status()  # Check if the request was successful

# Convert text response to a pandas DataFrame for the test dataset
new_test = pd.read_csv(StringIO(response_test.text), sep="\t")


In [10]:
# Convert train_dataset labels
train_dataset['class_label'] = [1 if label == 'Yes' else 0 for label in train_dataset['class_label']]

# Convert dev_dataset labels
dev_dataset['class_label'] = [1 if label == 'Yes' else 0 for label in dev_dataset['class_label']]

# Convert test_dataset labels
test_dataset['class_label'] = [1 if label == 'Yes' else 0 for label in test_dataset['class_label']]

# Convert labels to np.int32
train_dataset['class_label'] = train_dataset['class_label'].values.astype(np.int32)
dev_dataset['class_label'] = dev_dataset['class_label'].values.astype(np.int32)
test_dataset['class_label'] = test_dataset['class_label'].values.astype(np.int32)

In [11]:
train_dataset

,Sentence_id,Text,class_label
0,30313,And so I know that this campaign has caused so...,0
1,19099,"Now, let's balance the budget and protect Medi...",0
2,33964,I'd like to mention one thing.,0
3,16871,I must remind him the Democrats have controlle...,1
4,13150,And to take a chance uh - now be - and not mak...,0
...,...,...,...
22496,29631,"It would be squandered, too, believe me.",0
22497,7136,We're not allowed to vote on it.,1
22498,181,More Americans at work today than any time in ...,1
22499,12863,We indicated at that time that we were not goi...,1


In [12]:
train1 = train_dataset
train1

,Sentence_id,Text,class_label
0,30313,And so I know that this campaign has caused so...,0
1,19099,"Now, let's balance the budget and protect Medi...",0
2,33964,I'd like to mention one thing.,0
3,16871,I must remind him the Democrats have controlle...,1
4,13150,And to take a chance uh - now be - and not mak...,0
...,...,...,...
22496,29631,"It would be squandered, too, believe me.",0
22497,7136,We're not allowed to vote on it.,1
22498,181,More Americans at work today than any time in ...,1
22499,12863,We indicated at that time that we were not goi...,1


In [13]:
test_dataset

,Sentence_id,Text,class_label
0,37436,He's been a professor for a long time at a gre...,0
1,37440,There's no way they would give it up.,0
2,37463,They're able to charge women more for the same...,1
3,37470,"As far as a say is concerned, the people alrea...",0
4,37504,I am the Democratic Party right now.,0
...,...,...,...
313,40950,We had the best Black unemployment numbers in ...,1
314,40954,Success is going to bring us together.,0
315,40960,Your 401K's will go to hell and it'll be a ver...,0
316,40971,We're going to choose to move forward because ...,0


In [14]:
dev1 = dev_dataset
dev1

,Sentence_id,Text,class_label
0,26,"You know, I saw a movie - ""Crocodile Dundee.""",0
1,80,We're consuming 50 percent of the world's coca...,1
2,129,That answer was about as clear as Boston harbor.,0
3,131,Let me help the governor.,0
4,172,We've run up more debt in the last eight years...,1
...,...,...,...
1027,28958,He has promised a trillion dollars out of the ...,1
1028,28965,(LAUGHTER) I -- there's an old high school deb...,0
1029,29011,"Well, can I answer that?",0
1030,29060,I look forward to the final weeks of this camp...,0


###Task 2 - Defining Model and Tokenizer

In [15]:
# import torch
# from transformers import RobertaTokenizer, RobertaForSequenceClassification
from transformers import RobertaForSequenceClassification, RobertaTokenizer
# # Load the tokenizer and model
# tokenizer = RobertaTokenizer.from_pretrained('Dzeniks/roberta-fact-check')
# model = RobertaForSequenceClassification.from_pretrained('Dzeniks/roberta-fact-check')

# Write your code here

#*************************MODEL 1*************************************
# Define the tokenizer
tokenizer1 = BertTokenizer.from_pretrained('bert-base-uncased')
# Define the Language Model
model1 = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels = 2)
print(model1)

#*************************MODEL 2*************************************
from transformers import XLMRobertaTokenizer, XLMRobertaForSequenceClassification
model_name = "xlm-roberta-base"

model2 = XLMRobertaForSequenceClassification.from_pretrained(model_name, num_labels=2)
tokenizer2 = XLMRobertaTokenizer.from_pretrained(model_name)

#*************************MODEL 3*************************************

# from transformers import ElectraTokenizer, ElectraForSequenceClassification

# # Define the tokenizer
# tokenizer3 = ElectraTokenizer.from_pretrained('google/electra-base-discriminator')

# # Define the Language Model
# model3 = ElectraForSequenceClassification.from_pretrained('google/electra-base-discriminator', num_labels=2)
# print(model3)



/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

###Task 3 - Optimizer and Hyperparameters

In [16]:
training_args = TrainingArguments(
    output_dir="./con/Output/task1/output",
    num_train_epochs=5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    learning_rate=5e-5,
    weight_decay=0.004891290652279793,
    logging_dir="./con/Log/task1/logs",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_strategy="epoch"
)

##Task 4: Training Loop (for BERT)

In [17]:
from datasets import Dataset
def tokenize_function1(example):
    return tokenizer1(
        example["Text"],
        max_length=512,
        padding="max_length",
        truncation=True,
        return_tensors="pt",
    )

tokenized_train_Bert = Dataset.from_pandas(train1).map(tokenize_function1, batched=True)
tokenized_validation_Bert = Dataset.from_pandas(dev1).map(tokenize_function1, batched=True)

Map:   0%|          | 0/22501 [00:00<?, ? examples/s]

Map:   0%|          | 0/1032 [00:00<?, ? examples/s]

In [18]:
from datasets import Dataset
def tokenize_function2(example):
    return tokenizer2(
        example["Text"],
        max_length=512,
        padding="max_length",
        truncation=True,
        return_tensors="pt",
    )

tokenized_train_Ro = Dataset.from_pandas(train1).map(tokenize_function2, batched=True)
tokenized_validation_Ro = Dataset.from_pandas(dev1).map(tokenize_function2, batched=True)

Map:   0%|          | 0/22501 [00:00<?, ? examples/s]

Map:   0%|          | 0/1032 [00:00<?, ? examples/s]

In [19]:
# from datasets import Dataset
# def tokenize_function3(example):
#     return tokenizer3(
#         example["Text"],
#         max_length=512,
#         padding="max_length",
#         truncation=True,
#         return_tensors="pt",
#     )

# tokenized_train_alb = Dataset.from_pandas(train1).map(tokenize_function3, batched=True)
# tokenized_validation_alb = Dataset.from_pandas(dev1).map(tokenize_function3, batched=True)

In [20]:
tokenized_train_Bert
tkfor = tokenized_train_Bert.remove_columns(["Sentence_id", "Text"])
tkfor = tkfor.rename_column('class_label', 'labels')
tkfor

Dataset({
    features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 22501
})

In [21]:
tokenized_validation_Bert
vlfor = tokenized_validation_Bert.remove_columns(["Sentence_id", "Text"])
vlfor = vlfor.rename_column('class_label', 'labels')
vlfor

Dataset({
    features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 1032
})

In [22]:
tokenized_train_Ro
rotkfor = tokenized_train_Ro.remove_columns(["Sentence_id", "Text"])
rotkfor = rotkfor.rename_column('class_label', 'labels')
rotkfor

Dataset({
    features: ['labels', 'input_ids', 'attention_mask'],
    num_rows: 22501
})

In [23]:
tokenized_validation_Ro
rovlfor = tokenized_validation_Ro.remove_columns(["Sentence_id", "Text"])
rovlfor = rovlfor.rename_column('class_label', 'labels')
rovlfor

Dataset({
    features: ['labels', 'input_ids', 'attention_mask'],
    num_rows: 1032
})

In [24]:
# tokenized_train_alb
# altkfor = tokenized_train_alb.remove_columns(["Sentence_id", "Text"])
# altkfor = altkfor.rename_column('class_label', 'labels')
# altkfor

In [25]:
# tokenized_validation_alb
# alvlfor = tokenized_validation_alb.remove_columns(["Sentence_id", "Text"])
# alvlfor = alvlfor.rename_column('class_label', 'labels')
# alvlfor

In [26]:
data_collator_Bert = DataCollatorWithPadding(tokenizer=tokenizer1)
data_collator_Ro = DataCollatorWithPadding(tokenizer=tokenizer2)
# data_collator_alb = DataCollatorWithPadding(tokenizer=tokenizer3)

In [27]:
torch.cuda.empty_cache()

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model1.to(device)

# Define the Trainer
trainer = Trainer(
    model=model1,
    args=training_args,
    data_collator=data_collator_Bert,
    train_dataset=tkfor,
    eval_dataset=vlfor,
    )

# Train the model
train_Final_results1= trainer.train()

Epoch,Training Loss,Validation Loss
1,0.324400,0.090076
2,0.189300,0.088603
3,0.094000,0.135933
4,0.038600,0.208113


Epoch,Training Loss,Validation Loss
1,0.324400,0.090076
2,0.189300,0.088603
3,0.094000,0.135933
4,0.038600,0.208113
5,0.014000,0.213724


In [28]:
torch.cuda.empty_cache()

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model2.to(device)

# Define the Trainer
trainer = Trainer(
    model=model2,
    args=training_args,
    data_collator=data_collator_Ro,
    train_dataset=rotkfor,
    eval_dataset=rovlfor,
    )

# Train the model
train_Final_results2= trainer.train()

Epoch,Training Loss,Validation Loss
1,0.395100,0.144779
2,0.311000,0.105822
3,0.242200,0.078167
4,0.185200,0.094595


Epoch,Training Loss,Validation Loss
1,0.395100,0.144779
2,0.311000,0.105822
3,0.242200,0.078167
4,0.185200,0.094595
5,0.127500,0.142127


In [29]:
# torch.cuda.empty_cache()

# device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
# model3.to(device)

# # Define the Trainer
# trainer = Trainer(
#     model=model3,
#     args=training_args,
#     data_collator=data_collator_alb,
#     train_dataset=altkfor,
#     eval_dataset=alvlfor,
#     )

# # Train the model
# train_Final_results3= trainer.train()

In [30]:
!mkdir models1
!mkdir models2
# !mkdir models3

In [31]:
model1.save_pretrained("models1/")
model1 = BertForSequenceClassification.from_pretrained("models1/")

In [32]:
model2.save_pretrained("models2/")
model2 = XLMRobertaForSequenceClassification.from_pretrained("models2/")

In [33]:
# model3.save_pretrained("models3/")
# model3 = ElectraForSequenceClassification.from_pretrained("models3/")

In [34]:
log_history = trainer.state.log_history
log_history

[{'loss': 0.3951,
  'grad_norm': 22.412595748901367,
  'learning_rate': 4e-05,
  'epoch': 1.0,
  'step': 1407},
 {'eval_loss': 0.1447790265083313,
  'eval_runtime': 17.4231,
  'eval_samples_per_second': 59.232,
  'eval_steps_per_second': 3.731,
  'epoch': 1.0,
  'step': 1407},
 {'loss': 0.311,
  'grad_norm': 15.922758102416992,
  'learning_rate': 3e-05,
  'epoch': 2.0,
  'step': 2814},
 {'eval_loss': 0.10582150518894196,
  'eval_runtime': 17.4121,
  'eval_samples_per_second': 59.269,
  'eval_steps_per_second': 3.733,
  'epoch': 2.0,
  'step': 2814},
 {'loss': 0.2422,
  'grad_norm': 0.1540241837501526,
  'learning_rate': 2e-05,
  'epoch': 3.0,
  'step': 4221},
 {'eval_loss': 0.07816667854785919,
  'eval_runtime': 17.4046,
  'eval_samples_per_second': 59.295,
  'eval_steps_per_second': 3.735,
  'epoch': 3.0,
  'step': 4221},
 {'loss': 0.1852,
  'grad_norm': 45.474029541015625,
  'learning_rate': 1e-05,
  'epoch': 4.0,
  'step': 5628},
 {'eval_loss': 0.09459467977285385,
  'eval_runtime':

In [35]:
new_test

,Sentence_id,Text
0,35988,They said they were just going to get inspecto...
1,35991,"And from that point on, I've voted to -- I mov..."
2,36029,I sit on the Senate Armed Services Committee.
3,36043,We need to depend on all of our tools -- diplo...
4,36063,And that is -- and I don't know if my colleagu...
...,...,...
336,42558,And I grew up into a generation where we were ...
337,42569,"And the US Constitution, it is the strongest g..."
338,42571,That is what will win us the revolution of 2024.
339,42580,I understand the importance of the American dr...


In [36]:
test_lab = test_dataset.drop(test_dataset.columns[1], axis=1)

test_lab.head()

,Sentence_id,class_label
0,37436,0
1,37440,0
2,37463,1
3,37470,0
4,37504,0


In [37]:
# predictions = []
# threshold = 0.5

# model = model.to(device)

# for idx, row in new_test.iterrows():
#     arg_id = row['Sentence_id']
#     text = row['Text']
#     encoding = tokenizer(text, truncation=True, max_length=512, return_tensors='pt').to(device)
#     with torch.no_grad():
#         outputs = model(**encoding)

#     sigmoid = torch.nn.Sigmoid()
#     predictions_tensor = sigmoid(outputs.logits)

#     predicted_labels = []
#     for prob in predictions_tensor.flatten():
#         if prob >= threshold:
#           label = 0
#         else:
#           label = 1
#         predicted_labels.append(label)

#     predictions.append([arg_id, text] + predicted_labels)

# data = {'Sentence_id': [entry[0] for entry in predictions]}

# for i, label in enumerate(test_lab.columns[1:]):
#     data[label] = [entry[i+2] for entry in predictions]

# predictions_df = pd.DataFrame(data)
# predictions_df['class_label'] = ['Yes' if label == 1 else 'No' for label in predictions_df['class_label']]
# predictions_df['run_id'] = 'Model_1'

# predictions_df

# *****************************************************************

# import pandas as pd

# predictions = []
# threshold = 0.5

# model = model.to(device)

# for idx, row in test_dataset.iterrows():
#     arg_id = row['Sentence_id']
#     text = row['Text']
#     encoding = tokenizer(text, truncation=True, max_length=512, return_tensors='pt').to(device)
#     with torch.no_grad():
#         outputs = model(**encoding)

#     sigmoid = torch.nn.Sigmoid()
#     predictions_tensor = sigmoid(outputs.logits)

#     predicted_label = 0 if predictions_tensor[0, 0].item() >= threshold else 1
#     class_label = "Yes" if predicted_label == 1 else "No"

#     predictions.append([arg_id, class_label])  # Removed 'text' from here

# # Creating a DataFrame from the predictions list
# predicted_df = pd.DataFrame(predictions, columns=['Sentence_id', 'class_labels'])  # Removed 'Text' from here

# # Adding a new column 'run_id' with all entries as 'model_1'
# predicted_df['run_id'] = 'Model_1'

# # Saving DataFrame to a TSV file
# predicted_df


In [38]:
import pandas as pd

predictions = []
threshold = 0.5

model1 = model1.to(device)  # Assuming model1 is already defined and trained
model2 = model2.to(device)  # Assuming model2 is already defined and trained

for idx, row in test_dataset.iterrows():
    arg_id = row['Sentence_id']
    text = row['Text']

    # Tokenizing for model1
    encoding1 = tokenizer1(text, truncation=True, max_length=512, return_tensors='pt').to(device)
    with torch.no_grad():
        outputs1 = model1(**encoding1)
    sigmoid = torch.nn.Sigmoid()
    predictions_tensor1 = sigmoid(outputs1.logits)
    predicted_prob1 = predictions_tensor1[0, 0].item()

    # Tokenizing for model2
    encoding2 = tokenizer2(text, truncation=True, max_length=512, return_tensors='pt').to(device)
    with torch.no_grad():
        outputs2 = model2(**encoding2)
    predictions_tensor2 = sigmoid(outputs2.logits)
    predicted_prob2 = predictions_tensor2[0, 0].item()

    # Ensemble: Taking the average of predicted probabilities
    ensemble_prob = (predicted_prob1 + predicted_prob2) / 2

    predicted_label = 0 if ensemble_prob >= threshold else 1
    class_label = "Yes" if predicted_label == 1 else "No"

    predictions.append([arg_id, class_label])  # Removed 'text' from here

# Creating a DataFrame from the predictions list
predicted_df = pd.DataFrame(predictions, columns=['Sentence_id', 'class_labels'])  # Removed 'Text' from here

# Adding a new column 'run_id' with all entries as 'ensemble_model'
predicted_df['run_id'] = 'Ensemble_Model'

# Saving DataFrame to a TSV file
predicted_df


,Sentence_id,class_labels,run_id
0,37436,No,Ensemble_Model
1,37440,No,Ensemble_Model
2,37463,Yes,Ensemble_Model
3,37470,No,Ensemble_Model
4,37504,No,Ensemble_Model
...,...,...,...
313,40950,Yes,Ensemble_Model
314,40954,No,Ensemble_Model
315,40960,No,Ensemble_Model
316,40971,No,Ensemble_Model


In [39]:
!mkdir Output_File

In [40]:
output_path = "Output_File/output_predictions.tsv"
predicted_df.to_csv(output_path, sep='\t', index=False)

In [41]:
!python scorer/task_1.py -g=/content/dataset_converted/test_dataset.tsv -p=/content/Output_File/output_predictions.tsv --lang=english

INFO : Task 1: Checking format of file: /content/Output_File/output_predictions.tsv
INFO : Task 1: No issue found.
INFO : Started evaluating results for task-1_english ...
INFO : Reading gold labels from file /content/dataset_converted/test_dataset.tsv
INFO : Reading predicted labels from file /content/Output_File/output_predictions.tsv
acc: 0.9213836477987422, P:0.9882352941176471, R:0.7777777777777778, F1:0.8704663212435234


##Current Model's Score for english F1 on Bert-uncased and XLM-Roberta-base (positive class): 0.8704663212435234

##Previous Model's Score for english F1 on Bert-uncased (positive class): 0.7978723404255319

##Majority Baseline for english F1 (positive class): 0.0

##Random Baseline for english F1 (positive class): 0.30647291941875826

##Ngram Baseline for english F1 (positive class): 0.8526077097505668